In [ ]:
!pip install -U openai-whisper
!pip install srt
!pip install googletrans==4.0.0rc1

In [ ]:
import subprocess

# 音声データ抽出 (音声ファイル名変更)
subprocess.run(['ffmpeg', '-i', 'test.mp4', '-acodec', 'libmp3lame', '-ab', '256k', 'audio.mp3'])

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import logging
import time
from googletrans import Translator
from requests.exceptions import ReadTimeout
import srt


# 入力 SRT ファイル名
input_srt = 'sample.srt'

# ロガーの初期化
logging.basicConfig(level=logging.INFO)

# 翻訳機の初期化
translator = Translator()

def google_translate(text, src='en', dest='ja', max_attempts=5, sleep_time=0.2):
    """
    Google Translate を使用してテキストを翻訳する。

    Args:
        text (str): 翻訳対象のテキスト。
        src (str, optional): 翻訳元の言語コード。デフォルトは 'en' (英語)。
        dest (str, optional): 翻訳先の言語コード。デフォルトは 'ja' (日本語)。
        max_attempts (int, optional): 翻訳の最大試行回数。デフォルトは 5。
        sleep_time (float, optional): 試行間の待ち時間 (秒)。デフォルトは 0.2。

    Returns:
        googletrans.models.Translated: 翻訳結果オブジェクト。

    Raises:
        Exception: 最大試行回数を超えても翻訳に失敗した場合。
    """
    for attempt in range(max_attempts):
        try:
            return translator.translate(text, src=src, dest=dest)
        except Exception as e:
            logging.warning(f"Attempt {attempt+1}/{max_attempts}: {e}. Retrying...")
            if attempt < max_attempts - 1:
                time.sleep(sleep_time)
            else:
                raise Exception(f"翻訳に失敗しました ({max_attempts} 回の試行の後)。")


def read_srt_file(file_path):
    """
    SRT ファイルを読み込む。

    Args:
        file_path (str): SRT ファイルのパス。

    Returns:
        str: SRT ファイルの内容。
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def write_srt_file(file_path, content):
    """
    SRT ファイルに書き込む。

    Args:
        file_path (str): SRT ファイルのパス。
        content (str): 書き込む SRT ファイルの内容。
    """
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(content)

async def translate_subtitle_batch(batch, src='en', dest='ja'):
    """
    字幕のバッチを翻訳する。

    Args:
        batch (list[srt.Subtitle]): 翻訳対象の字幕オブジェクトのリスト。
        src (str, optional): 翻訳元の言語コード。デフォルトは 'en' (英語)。
        dest (str, optional): 翻訳先の言語コード。デフォルトは 'ja' (日本語)。
    """
    results = [google_translate(subtitle.content, src, dest) for subtitle in batch]
    for subtitle, translated in zip(batch, results):
        subtitle.content = translated.text

async def translate_subtitles(srt_data, batch_size=10, src='en', dest='ja', sleep_time=0.2):
    """
    SRT ファイルの字幕を翻訳する。

    Args:
        srt_data (str): SRT ファイルの内容。
        batch_size (int, optional): 翻訳する字幕のバッチサイズ。デフォルトは 10。
        src (str, optional): 翻訳元の言語コード。デフォルトは 'en' (英語)。
        dest (str, optional): 翻訳先の言語コード。デフォルトは 'ja' (日本語)。
        sleep_time (float, optional): バッチ翻訳後の待ち時間 (秒)。デフォルトは 0.2。

    Returns:
        list[srt.Subtitle]: 翻訳された字幕オブジェクトのリスト。
    """
    subtitles = list(srt.parse(srt_data))
    count = 1
    for i in range(0, len(subtitles), batch_size):
        batch = subtitles[i:i + batch_size]
        await translate_subtitle_batch(batch, src, dest)
        for j in range(len(batch)):
            logging.info(f"{count}/{len(subtitles)} Translated: {i+j+1}. {subtitles[i+j].content}")
            count += 1
        time.sleep(sleep_time)
    return subtitles

async def translate_srt(input_file, output_file, batch_size=10):
    """
    SRT ファイルを翻訳する。

    Args:
        input_file (str): 入力 SRT ファイルのパス。
        output_file (str): 出力 SRT ファイルのパス。
        batch_size (int, optional): 翻訳する字幕のバッチサイズ。デフォルトは 10。
    """
    srt_data = read_srt_file(input_file)
    translated_subtitles = await translate_subtitles(srt_data, batch_size)
    translated_srt_content = srt.compose(translated_subtitles)
    write_srt_file(output_file, translated_srt_content)

# 翻訳の実行
await translate_srt(input_srt, 'translated_file_google.srt', batch_size=10)

def add_line_breaks(text, max_length=20, max_lines=2):
    """
    句読点と長さに応じてテキストに改行を追加する。

    Args:
        text (str): 改行を追加するテキスト。
        max_length (int, optional): 1行の最大文字数。デフォルトは 20。
        max_lines (int, optional): 最大行数。デフォルトは 2。

    Returns:
        str: 改行が追加されたテキスト。
    """
    result = ""
    current_line = ""
    lines = []

    for char in text:
        if char == "、" or char == "。":
            current_line += char
            if len(current_line) > max_length or char == "。":
                lines.append(current_line)
                current_line = ""
        else:
            current_line += char

    if current_line:
        lines.append(current_line)

    # 行数を制限する
    while len(lines) > max_lines:
        line1 = lines.pop(0)
        line2 = lines.pop(0)
        combined_line = line1 + line2
        lines.insert(0, combined_line)

    return "\n".join(lines)

def add_line_breaks_srt(srt_text):
    """
    SRT ファイルの字幕に改行を追加する。

    Args:
        srt_text (str): SRT ファイルの内容。

    Returns:
        str: 改行が追加された SRT ファイルの内容。
    """
    lines = srt_text.split("\n")
    result = ""

    for i, line in enumerate(lines):
        if line.strip().isdigit():
            result += line
        elif "-->" in line:
            result += line
        else:
            result += add_line_breaks(line) + "\n"

        # 空行を追加 (字幕間の空白)
        if i < len(lines) - 1 and lines[i + 1] != "" and line != "":
            result += "\n"

    return result.strip()

# 翻訳された SRT ファイルを読み込む
with open('translated_file_google.srt', 'r', encoding='utf-8') as file:
    srt_text = file.read()

# 改行を追加する
result = add_line_breaks_srt(srt_text)

# 結果をファイルに書き込む
with open("translated_file_result.srt", "w", encoding='utf-8') as file:
    file.write(result)

In [ ]:
import subprocess

video = input_srt.replace(".srt", ".mp4")

subprocess.run([
    'ffmpeg',
    '-hwaccel', 'auto', 
    '-i', f"{video}",
    '-vf', "subtitles=translated_file_google.srt:force_style='FontName=Helvetica,FontSize=11'",
    '-c:v', 'h264_amf', 
    '-c:a', 'copy', 
    f'{video.replace(".mp4", "_jp.mp4")}' 
])
